In [1]:
import mysql.connector
import base64
import time
import numpy as np
import pickle
import os

In [28]:
dataBase = mysql.connector.connect(
    host="washington.uww.edu",
    user="stremmeltr18",
    passwd=base64.b64decode(b'dHM1NjEy').decode("utf-8"),
    database="manga_rec"
)
myCursor = dataBase.cursor()
myCursor.execute("select * from manga;")
manga = [x for x in myCursor]
for i in range(len(manga)):
    manga[i] = list(manga[i])
    del manga[i][3]  # drop description for ease of viewing output

In [19]:
print(manga[4])

[4, 33, 'Monster', 'description', 1994, 162, 'Finished', 'Drama|Mystery', 'Adult Cast|Psychological', 'Seinen', 'https://cdn.myanimelist.net/r/42x62/images/characters/4/316522.jpg?s=0e1d82e64cdde251b0c7f38c99efdeb4']


In [13]:
maxPopularity = 27691
minPopularity = 1
maxReleaseDate = 2022
minReleaseDate = 1946
maxChapterCount = 6477
minChapterCount = 1

In [16]:
encodedManga = []
for mangaInstance in manga:
    instanceData = [mangaInstance[0], mangaInstance[1], mangaInstance[4] if mangaInstance[4] is not None else 2008,
                        mangaInstance[5] if mangaInstance[5] is not None else 40]
    instanceData[1] = (instanceData[1]-minPopularity)/(maxPopularity-minPopularity)
    instanceData[2] = (instanceData[2]-minReleaseDate)/(maxReleaseDate-minReleaseDate)
    instanceData[3] = (instanceData[3]-minChapterCount)/(maxChapterCount-minChapterCount)
    encodedManga.append(instanceData)
for i in range(10):
    print(encodedManga[i])

[0, 3.611412062116287e-05, 0.5657894736842105, 0.05852378011117974]
[1, 0.0009028530155290719, 0.7631578947368421, 0.014669549104385423]
[2, 7.222824124232574e-05, 0.6710526315789473, 0.006022235948116121]
[3, 0.000577825929938606, 0.6842105263157895, 0.05033971587399629]
[4, 0.001155651859877212, 0.631578947368421, 0.024861025324274245]
[5, 0.0006139400505597688, 0.7236842105263158, 0.01775787523162446]
[6, 0.0019501625135427952, 0.8947368421052632, 0.006022235948116121]
[7, 0.00036114120621162876, 0.8026315789473685, 0.022544780728844967]
[8, 0.0023835319609967496, 0.5789473684210527, 0.04246448424953675]
[9, 0.0007583965330444204, 0.7763157894736842, 0.006022235948116121]


In [18]:
for i in range(len(encodedManga)):
    if encodedManga[3] is None:
        print(i, encodedManga[i])

In [30]:
#make manga_id -> manga_index np array
get_manga_array_index = np.full((10000), -1, dtype=int)
print(get_manga_array_index.shape)
print(get_manga_array_index)
for i in range(len(manga)):
    get_manga_array_index[manga[i][0]] = i
print(get_manga_array_index)

(10000,)
[-1 -1 -1 ... -1 -1 -1]
[   0    1    2 ... 9871 9872 9873]


In [2]:
def convertRating(ratingTuple):
    mangaId = ratingTuple[2]
    status = ratingTuple[3]
    originalRating = ratingTuple[4]

    if originalRating is not None:
        if originalRating >= 7:
            return mangaId, 5  # 5
        if originalRating >= 5:
            return mangaId, 3  # 3
        else:  # if rating < 5
            return mangaId, 2  # 2
    if status == 'reading' or status == 'completed':
        return mangaId, 4  # 4
    if status == 'plan_to_read':
        return mangaId, 4  # 4
    if status == 'on_hold':
        return mangaId, 3  # 3
    if status == 'dropped':
        return mangaId, 2  # 2
    if status == 'not interested':
        return mangaId, 1
    return mangaId, 4  # 4 

def convertRating_forKnn(ratingTuple):
    convertedRating = convertRating(ratingTuple)
    if ratingTuple[1] is None:  # (new note) error checking
        print("hi")
    if convertedRating is None:  # (new note) error checking
        print(ratingTuple)
    return ratingTuple[1], convertedRating[0], convertedRating[1]  # userId, mangaId, convertedRatingValue

In [31]:
myCursor.execute("select * from ratings;")
ratings = myCursor.fetchall()  # [x for x in myCursor]
convertedRatings = [convertRating_forKnn(x) for x in ratings]

In [33]:
print(len(convertedRatings))
np.save('convertedRatings', convertedRatings)

2546362


In [64]:
def getStatusSet():
    return ['On Hiatus', 'Finished', 'Publishing', 'Discontinued']
def getGenreSet():
    return ['Adventure', 'Comedy', 'Slice of Life', 'Boys Love', 'Sci-Fi', 'Action', 'Horror', 'Suspense', 'Girls Love',
            'Gourmet', 'Sports', 'Avant Garde', 'Supernatural', 'Fantasy', 'Romance', 'Ecchi', 'Drama', 'Mystery']
def getThemeSet():
    return ['Historical', 'Time Travel', 'Visual Arts', 'Military', 'Love Polygon', 'Mecha', 'Martial Arts', 'Racing',
            'Samurai', 'Strategy Game', 'CGDCT', 'Mythology', 'High Stakes Game', 'Idols (Male)', 'Reincarnation',
            'Pets', 'Team Sports', 'Workplace', 'Isekai', 'Gag Humor', 'Memoir', 'Harem', 'Villainess', 'Detective',
            'Performing Arts', 'Reverse Harem', 'Childcare', 'Otaku Culture', 'Mahou Shoujo', 'Anthropomorphic',
            'Survival', 'Magical Sex Shift', 'Music', 'Delinquents', 'Organized Crime', 'Adult Cast', 'Medical',
            'Showbiz', 'Crossdressing', 'Gore', 'Psychological', 'School', 'Combat Sports', 'Parody',
            'Romantic Subtext', 'Space', 'Iyashikei', 'Video Game', 'Educational', 'Vampire', 'Super Power']
def getDemographicSet():
    return ['Kids', 'Seinen', 'Shoujo', 'Josei', 'Shounen']

def encodeManga(manga):
    mangaEncoded = []
    rawFeaturesThatWereNormalized = []
    statusSet = getStatusSet()
    genreSet = getGenreSet()
    themeSet = getThemeSet()
    demographicSet = getDemographicSet()
    maxPopularity = 27691
    minPopularity = 1
    maxReleaseDate = 2022
    minReleaseDate = 1946
    maxChapterCount = 6477
    minChapterCount = 1
    for mangaInstance in manga:
        # make fill release date for None type,
        # uses 2008 as fill for when release date is none. 2008 is average release data and there are only 100 manga...
        # with no release date
        # rn chapterCount is set to 40 if none, (avg is 40) ************************************************************
        instanceData = [mangaInstance[0], mangaInstance[1], mangaInstance[4] if mangaInstance[4] is not None else 2008,
                        mangaInstance[5] if mangaInstance[5] is not None else 40]
        rawFeaturesThatWereNormalized.append(instanceData.copy())
        # normalize release date, chapter count, and popularity between 0 and 1 (new)
        instanceData[1] = (instanceData[1] - minPopularity) / (maxPopularity - minPopularity)
        instanceData[2] = (instanceData[2] - minReleaseDate) / (maxReleaseDate - minReleaseDate)
        instanceData[3] = (instanceData[3] - minChapterCount) / (maxChapterCount - minChapterCount)
        
        mangaInstanceStatusSet = mangaInstance[6]
        for i in range(len(statusSet)):
            if statusSet[i] == mangaInstanceStatusSet:
                instanceData.append(1)
            else:
                instanceData.append(0)
        mangaInstanceGenreSet = mangaInstance[7].split('|') if mangaInstance[7] is not None else []
        for i in range(len(genreSet)):
            if genreSet[i] in mangaInstanceGenreSet:
                instanceData.append(1)
            else:
                instanceData.append(0)
        mangaInstanceThemeSet = mangaInstance[8].split('|') if mangaInstance[8] is not None else []
        for i in range(len(themeSet)):
            if themeSet[i] in mangaInstanceThemeSet:
                instanceData.append(1)
            else:
                instanceData.append(0)
        mangaInstanceDemographicSet = mangaInstance[9].split('|') if mangaInstance[9] is not None else []
        for i in range(len(demographicSet)):
            if demographicSet[i] in mangaInstanceDemographicSet:
                instanceData.append(1)
            else:
                instanceData.append(0)

        mangaEncoded.append(instanceData)

    return mangaEncoded, rawFeaturesThatWereNormalized

In [65]:
dataBase = mysql.connector.connect(
    host="washington.uww.edu",
    user="stremmeltr18",
    passwd=base64.b64decode(b'dHM1NjEy').decode("utf-8"),
    database="manga_rec"
)
myCursor = dataBase.cursor()

# create one hot encoded (and other data alterations) matrix of manga table
# possible include no_genres/no_themes column (i dont think it would be good but idk)
myCursor.execute("select * from manga;")
manga = [x for x in myCursor]
encodedManga, rawFeaturesThatWereNormalized = encodeManga(manga)

In [45]:
for i in range(20):
    print(rawFeaturesThatWereNormalized[i])
    print(encodedManga[i])

[0, 2, 1989, 380]
[0, 3.611412062116287e-05, 0.5657894736842105, 0.05852378011117974, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[1, 26, 2004, 96]
[1, 0.0009028530155290719, 0.7631578947368421, 0.014669549104385423, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1]
[2, 3, 1997, 40]
[2, 7.222824124232574e-05, 0.6710526315789473, 0.006022235948116121, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]
[3, 17, 1998, 327]
[3, 0.00057782592993

In [70]:
for i in range(len(manga)):
    manga[i] = list(manga[i])
    del manga[i][3]  # drop description for ease of viewing output

In [55]:
np.save('get_manga_array_index', get_manga_array_index)

In [71]:
with open('manga_raw.pkl', 'wb') as f:
    pickle.dump(manga, f)

In [54]:
get_manga_array_index = np.full(shape=10000, fill_value=-1, dtype=int)
for i in range(len(manga)):
    get_manga_array_index[manga[i][0]] = i

In [60]:
print(encodedManga[0][4:])
print(rawFeaturesThatWereNormalized[0][:4])

[1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[0, 2, 1989, 380]


In [2]:
imagesMap = os.listdir('H:/manga_rec/manga_rec_back_end/manga_images/images')

In [3]:
np.save('imagesMap', imagesMap)

In [ ]:
dataBase = mysql.connector.connect(
    host="washington.uww.edu",
    user="stremmeltr18",
    passwd=base64.b64decode(b'dHM1NjEy').decode("utf-8"),
    database="manga_rec"
)
myCursor = dataBase.cursor()

convertRati